In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime
from datetime import date, timedelta
today = date.today()

In [2]:
d1 = today.strftime("%Y-%m-%d")
end_date = d1
d2 = date.today() - timedelta(days=5000)
d2 = d2.strftime("%Y-%m-%d")
start_date = d2

In [3]:
df = yf.download('NFLX',
                 start=start_date,
                 end = end_date,
                 progress=False
                 )
df['Date'] = df.index
df = df[["Date","Open","High","Low","Close","Adj Close","Volume"]]
df.reset_index(drop=True, inplace=True)
print(df.tail())

           Date        Open        High         Low       Close   Adj Close  \
3439 2023-08-22  409.000000  415.739990  407.399994  413.170013  413.170013   
3440 2023-08-23  418.399994  437.019989  417.709991  427.549988  427.549988   
3441 2023-08-24  425.410004  427.589996  406.170013  406.929993  406.929993   
3442 2023-08-25  412.000000  419.250000  407.559998  416.029999  416.029999   
3443 2023-08-28  418.040009  419.829987  413.269989  418.059998  418.059998   

       Volume  
3439  3603300  
3440  8174700  
3441  5687800  
3442  5386800  
3443  2926000  


In [4]:
import plotly.graph_objects as go
figure = go.Figure(data=[go.Candlestick(x=df['Date'],
                                  open=df['Open'],
                                  high=df['High'],
                                  low=df['Low'],
                                  close=df['Close'],
                                  )])
figure.update_layout(title = "Netflix Stock Price Analysis",
                     xaxis_rangeslider_visible=False)
figure.show()


In [5]:
df.corr()['Close']

<ipython-input-5-8695f15c585d>:1: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



Open         0.999537
High         0.999797
Low          0.999781
Close        1.000000
Adj Close    1.000000
Volume      -0.477398
Name: Close, dtype: float64

## Train LSTM Model

In [6]:
x = df[["Open","High","Low","Volume"]]
y = df['Close']

X = x.to_numpy()
y = y.to_numpy()
y = y.reshape(-1,1)


In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=27)

In [8]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape= (X_train.shape[1], 1)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 4, 128)            66560     
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 25)                1625      
                                                                 
 dense_1 (Dense)             (None, 1)                 26        
                                                                 
Total params: 117,619
Trainable params: 117,619
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(optimizer='adam',loss='mean_squared_error')
model.fit(X_train,y_train, batch_size=1, epochs=30)

Epoch 1/30
2755/2755 [==============================] - 23s 5ms/step - loss: 8022.9072
Epoch 2/30
2755/2755 [==============================] - 13s 5ms/step - loss: 545.9431
Epoch 3/30
2755/2755 [==============================] - 14s 5ms/step - loss: 346.9297
Epoch 4/30
2755/2755 [==============================] - 13s 5ms/step - loss: 408.6594
Epoch 5/30
2755/2755 [==============================] - 13s 5ms/step - loss: 440.6842
Epoch 6/30
2755/2755 [==============================] - 13s 5ms/step - loss: 456.0440
Epoch 7/30
2755/2755 [==============================] - 14s 5ms/step - loss: 363.2841
Epoch 8/30
2755/2755 [==============================] - 14s 5ms/step - loss: 349.1722
Epoch 9/30
2755/2755 [==============================] - 13s 5ms/step - loss: 283.0423
Epoch 10/30
2755/2755 [==============================] - 14s 5ms/step - loss: 233.3965
Epoch 11/30
2755/2755 [==============================] - 13s 5ms/step - loss: 264.7390
Epoch 12/30
2755/2755 [============================

In [10]:
# features = [Open, High, Low, Adj Close, Volume]
features = np.array([[401.970001,427.700012, 398.200012, 20047500 ]])
model.predict(features)

1/1 [==============================] - 1s 629ms/step


array([[404.76593]], dtype=float32)